# Kaggle 데이터 다운로드

방법은 https://www.kaggle.com/general/74235 문서를 참조함

아래의 코드는 실행하기 전에

- Kaggle에 계정이 있어야 하고, 
- dogs-vs-cats의 Data 섹션에서 data사용에 대한 동의를 미리 해야 함. 
- 그리고 API인증 토큰 kaggle.json을 다운로드 받아놔야 

함.



## 모듈 kaggle 설치

In [0]:
! pip install -q kaggle

## 토큰 준비

인증 정보가 담긴 토큰 파일을 업로드하고 특정 디렉토리에 위치 시킴.

저 파일은 kaggle 홈페이지에서 다운받은 것.


In [4]:
!rm -rf kaggle*.json

from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dhrim0000","key":"e9167848d5a79eb1abcb88d6655b585e"}'}

In [0]:
! rm -rf ~/.kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
 ! kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
new-york-state/nys-environmental-remediation-sites          NYS Environmental Remediation Sites                   7MB  2019-08-05 08:25:29           1737  
new-york-city/new-york-city-current-job-postings            New York City Current Job Postings                    3MB  2019-08-04 22:04:44           5605  
google/google-landmarks-dataset                             Google-Landmarks Dataset                            107MB  2019-08-04 12:00:25          10466  
eliasdabbas/search-engine-results-flights-tickets-keywords  Search Engine Results - Flights & Tickets Keywords   22MB  2019-08-01 11:51:13           1306  
AnalyzeBoston/crimes-in-boston                              Crim

## 실제 데이터 다운로드

In [8]:
! kaggle competitions download -c 'dogs-vs-cats'

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 31.5MB/s]
 96% 260M/271M [00:02<00:00, 142MB/s]
100% 271M/271M [00:02<00:00, 113MB/s]
 99% 539M/543M [00:04<00:00, 154MB/s]
100% 543M/543M [00:04<00:00, 124MB/s]


In [9]:
!ls -al

total 833976
drwxr-xr-x 1 root root      4096 Aug  5 12:13 .
drwxr-xr-x 1 root root      4096 Aug  5 11:29 ..
drwxr-xr-x 1 root root      4096 Aug  1 16:08 .config
-rw-r--r-- 1 root root        65 Aug  5 12:13 kaggle.json
drwxr-xr-x 1 root root      4096 Aug  1 16:08 sample_data
-rw-r--r-- 1 root root     88903 Aug  5 12:13 sampleSubmission.csv
-rw-r--r-- 1 root root 284321224 Aug  5 12:13 test1.zip
-rw-r--r-- 1 root root 569546721 Aug  5 12:13 train.zip


In [0]:
!unzip train.zip
!unzip test1.zip

## 폴더 구조로 train 데이터 준비

다음 구조로 변경한다
```
dogs_cats
├── train
│   ├── cat
│   └── dog
└── valid
    ├── cat
    └── dog

```

In [11]:
!ls -al train | wc 

  25003  225020 1402884


In [0]:
! mkdir dogs_cats
! mkdir -p dogs_cats/train/dog
! mkdir -p dogs_cats/train/cat
! mkdir -p dogs_cats/valid/dog
! mkdir -p dogs_cats/valid/cat

In [13]:
!apt install tree
!tree dogs_cats

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (79.9 kB/s)
Selecting previously unselected package tree.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
dogs_cats
├── train
│   ├── cat
│   └── dog
└── valid
    ├── cat
    └── dog

6 directories, 0 files


In [14]:
!ls -al train/dog.9* | wc

   1111    9999   67648


In [0]:
!mv train/dog.9* dogs_cats/valid/dog/
!mv train/dog.8* dogs_cats/valid/dog/

!mv train/dog.* dogs_cats/train/dog/


!mv train/cat.9* dogs_cats/valid/cat/
!mv train/cat.8* dogs_cats/valid/cat/

!mv train/cat.* dogs_cats/train/cat/



In [16]:
!ls -asl dogs_cats/train/dog/ | wc
!ls -asl dogs_cats/train/cat/ | wc

!ls -asl dogs_cats/valid/dog/ | wc
!ls -asl dogs_cats/valid/cat/ | wc


  10281  102802  618428
  10281  102802  618428
   2225   22242  128737
   2225   22242  128737


## test 데이터 준비

ImageDataGenerator를 사용할면, test밑에 sub 폴더 하나가 더 있어야 한다.

```
test/
  unknown/
    0000.jpg
    0001.jpg
    ...
```

In [36]:
!ls -al test1/*.jpg | wc

  12500  112500  713894


In [0]:
!mkdir -p test/unknown
!cp test1/* test/unknown/

In [35]:
!ls -al test/unknown/*.jpg | wc

  12500  112500  801394


# 학습 실행

In [17]:
import os

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import VGG16


conv_layers = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers.trainable = False

model = models.Sequential()

model.add(conv_layers)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])


data_aug_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=vgg16.preprocess_input
)

data_no_aug_generator = ImageDataGenerator(
      preprocessing_function=vgg16.preprocess_input
)

train_data_generator = data_aug_generator.flow_from_directory(
      "dogs_cats/train", # MODIFIED
      target_size=(224,224),
      batch_size=64,
      class_mode='sparse'
)

valid_data_generator = data_no_aug_generator.flow_from_directory(
      "dogs_cats/valid", # MODIFIED
      target_size=(224,224),
      class_mode='sparse'
)


model.fit_generator(
      train_data_generator,
      validation_data=valid_data_generator,
      validation_steps=5,
      steps_per_epoch=train_data_generator.samples/64, # data_size / batch_size    
      epochs=10
)


y_ = model.predict_generator(
      test_data_generator,
      steps=test_data_generator.samples/64
)

custom_labels = list(test_data_generator.class_indices.keys()) # ADDED
predicted = np.argmax(y_, axis=1)
print(predicted[0], custom_labels[predicted[0]])

W0805 12:16:15.319836 140465958664064 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


58892288/58889256 [==============================] - 2s 0us/step
Found 20556 images belonging to 2 classes.
Found 4444 images belonging to 2 classes.
Epoch 1/10
322/321 [==============================] - 258s 803ms/step - loss: 9.2304 - acc: 0.9533 - val_loss: 0.2277 - val_acc: 0.9812
Epoch 2/10
322/321 [==============================] - 245s 761ms/step - loss: 0.6200 - acc: 0.9711 - val_loss: 0.1335 - val_acc: 0.9875
Epoch 3/10
322/321 [==============================] - 246s 765ms/step - loss: 0.3576 - acc: 0.9757 - val_loss: 0.0723 - val_acc: 0.9875
Epoch 4/10
322/321 [==============================] - 245s 762ms/step - loss: 0.2621 - acc: 0.9788 - val_loss: 4.9076e-05 - val_acc: 1.0000
Epoch 5/10
322/321 [==============================] - 247s 767ms/step - loss: 0.2829 - acc: 0.9805 - val_loss: 0.0886 - val_acc: 0.9812
Epoch 6/10
322/321 [==============================] - 246s 765ms/step - loss: 0.2529 - acc: 0.9810 - val_loss: 2.6352e-05 - val_acc: 1.0000
Epoch 7/10
322/321 [======

In [0]:
model.save("kaggle_dogs_and_cats.h5")

# 예측하기

대상 파일은 test1 폴더에 있다.

당연히 클래스에 해당하는 서브폴더나 이름으로 구별할 수 없다.

ImageDataGenerator가 읽을 수 있도록 test/unknown에 가져다 두었다.
```
test\
  unknown\
    10000.jpg
    10001.jpg
    10002.jpg
```

제출할 파일의 샘플은 sampleSubmission.csv
```
id,label
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
```

예측 실행하고 위 파일과 같은 포멧으로 submission.csv 파일을 생성하면 된다.

In [69]:
!ls -al test1 | head -10

total 304272
drwxr-xr-x 2 root root 282624 Sep 20  2013 .
drwxr-xr-x 1 root root   4096 Aug  5 13:20 ..
-rw-r--r-- 1 root root  54902 Sep 20  2013 10000.jpg
-rw-r--r-- 1 root root  21671 Sep 20  2013 10001.jpg
-rw-r--r-- 1 root root  28895 Sep 20  2013 10002.jpg
-rw-r--r-- 1 root root  40146 Sep 20  2013 10003.jpg
-rw-r--r-- 1 root root  21252 Sep 20  2013 10004.jpg
-rw-r--r-- 1 root root   8888 Sep 20  2013 10005.jpg
-rw-r--r-- 1 root root  11972 Sep 20  2013 10006.jpg


In [44]:
!head -10 sampleSubmission.csv

id,label
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


## 예측 실행

In [38]:
test_data_generator = data_no_aug_generator.flow_from_directory(
      "test",
      target_size=(224,224),
      class_mode='sparse',
      shuffle=False,
      batch_size=1
)


Found 12500 images belonging to 1 classes.


In [0]:
sample_count = len(test_data_generator.filenames)
predict = model.predict_generator(test_data_generator, steps=sample_count)

## 예측된 결과를 담고 파일생성할 pandas DataFrame

In [0]:
import pandas as pd

test_df = pd.DataFrame({
    'filename': test_data_generator.filenames
})
test_df['category'] = np.argmax(predict, axis=-1)

In [42]:
test_df.head()

,filename,category
0,unknown/1.jpg,1
1,unknown/10.jpg,0
2,unknown/100.jpg,0
3,unknown/1000.jpg,1
4,unknown/10000.jpg,1


In [48]:
test_df.head()

,filename,category
0,unknown/1.jpg,1
1,unknown/10.jpg,0
2,unknown/100.jpg,0
3,unknown/1000.jpg,1
4,unknown/10000.jpg,1


In [66]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('/').str[1].str.split('.').str[0].astype(int)
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df = submission_df.sort_values(['id'])
submission_df.to_csv('submission.csv', index=False)

submission_df.head()

,id,label
0,1,1
3612,2,1
4723,3,1
5834,4,1
6945,5,0


In [67]:
!head -10 submission.csv

id,label
1,1
2,1
3,1
4,1
5,0
6,0
7,0
8,0
9,0
